# 📄 정산 문서 자동 처리 시스템

## 기능
1. 폴더별 PDF 스캔 및 분류 (규칙 기반)
2. 문서 순서대로 PDF 합본 생성
3. 필수 문서 누락 체크
4. 구글 시트용 결과 데이터 생성

## 사용법
셀을 위에서부터 순서대로 실행하세요!

## 1단계: 환경 설정

In [20]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# 라이브러리 설치
!pip install PyPDF2 Pillow openai -q

print("라이브러리 설치 완료")

라이브러리 설치 완료


In [22]:
# 라이브러리 임포트
import re
from pathlib import Path
from typing import Dict, List, Optional, Tuple
from PyPDF2 import PdfMerger, PdfReader
from PIL import Image
from datetime import datetime
import pandas as pd
import io

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


## 2단계: 설정 (여기만 수정하세요!)

In [23]:
# ⚙️ 설정 (여기만 수정하세요!)

class Config:
    """중앙 집중식 설정 관리"""

    # 📁 경로 설정
    BASE_PATH = "경로 입력"

    # 🤖 OpenAI API 설정
    OPENAI_API_KEY = "KEY 입력"  # ⬅️ 여기에 OpenAI API 키를 입력하세요!
    USE_GPT_CLASSIFICATION = True  # True = GPT 사용, False = 정규식 사용

    # 🧪 테스트 모드
    TEST_MODE = False  # False = 전체 폴더 처리

    # 📊 구글 시트 설정
    SHEET_URL = "https://docs.google.com/spreadsheets/d/YOUR_SHEET_ID/edit"  # 구글 시트 URL 입력

    # 🔧 처리 옵션
    SKIP_EXISTING_MERGED = True
    CLEANUP_TEMP_FILES = True

    # 📝 필수 문서 정의 (실제 필수 서류 목록)
    REQUIRED_DOCS = {
        "용역비": ["전자세금계산서", "견적서", "계약서", "이체확인증", "사업자등록증", "통장사본", "자문보고서"],
        "사업추진비": ["매출전표", "영수증", "회의록"],
        "일반수용비": ["원천징수영수증", "이체확인증", "이력서", "통장사본", "신분증", "비용지급확인서", "자문보고서"],
        "행사비": ["매출전표"],
    }

    # 📄 문서 순서 정의 (합본 시 사용)
    DOCUMENT_ORDER = {
        "용역비": ["전자세금계산서", "견적서", "계약서", "이체확인증", "사업자등록증", "통장사본", "자문보고서", "기타"],
        "사업추진비": ["매출전표", "영수증", "회의록", "기타"],
        "일반수용비": ["원천징수영수증", "이체확인증", "이력서", "통장사본", "신분증", "비용지급확인서", "자문보고서", "강의자료", "기타"],
        "행사비": ["매출전표", "기타"],
    }

print("✅ 설정 완료!")


✅ 설정 완료!


## 3단계: 핵심 클래스 정의

In [24]:
class FileConverter:
    """이미지 파일을 PDF로 변환"""

    SUPPORTED_IMAGES = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff'}

    @classmethod
    def is_image(cls, file_path: Path) -> bool:
        """이미지 파일인지 확인"""
        return file_path.suffix.lower() in cls.SUPPORTED_IMAGES

    @classmethod
    def convert_image_to_pdf(cls, image_path: Path, output_path: Path) -> bool:
        """
        이미지를 PDF로 변환

        Args:
            image_path: 원본 이미지 경로
            output_path: 출력 PDF 경로

        Returns:
            bool: 변환 성공 여부
        """
        try:
            # 이미지 열기
            img = Image.open(image_path)

            # RGBA 모드를 RGB로 변환 (PDF는 RGBA 미지원)
            if img.mode in ('RGBA', 'LA', 'P'):
                # 흰색 배경 생성
                background = Image.new('RGB', img.size, (255, 255, 255))
                if img.mode == 'P':
                    img = img.convert('RGBA')
                background.paste(img, mask=img.split()[-1] if img.mode in ('RGBA', 'LA') else None)
                img = background
            elif img.mode != 'RGB':
                img = img.convert('RGB')

            # 이미지 크기 조정 (A4 비율에 맞게)
            # A4 = 210mm x 297mm = 2480px x 3508px at 300dpi
            max_width, max_height = 2480, 3508
            img.thumbnail((max_width, max_height), Image.Resampling.LANCZOS)

            # PDF로 저장
            img.save(output_path, 'PDF', resolution=100.0, quality=95)

            return True

        except Exception as e:
            print(f"  ⚠️  이미지 변환 실패 ({image_path.name}): {str(e)}")
            return False

print("✅ FileConverter 정의 완료")

✅ FileConverter 정의 완료


In [25]:
# GPT 기반 문서 추출 및 분류
from openai import OpenAI
import json

class GPTDocumentExtractor:
    """GPT-4o-mini 기반 문서 추출 및 분류"""

    def __init__(self, api_key: str):
        self.client = OpenAI(api_key=api_key)
        self.model = "gpt-4o-mini"
        self.call_count = 0

    def classify_file(self, filename: str) -> str:
        """파일명으로 문서 타입 분류 (GPT)"""
        self.call_count += 1

        prompt = f"""다음 파일명을 보고 문서 타입을 분류하세요.

파일명: {filename}

가능한 문서 타입:
- 이체확인증
- 전자세금계산서
- 원천징수영수증
- 매출전표
- 사업자등록증
- 계약서
- 견적서
- 신분증
- 통장사본
- 이력서
- 비용지급확인서
- 자문보고서
- 강의자료
- 회의록
- 방명록
- 영수증
- 기타

위 목록 중 하나만 출력하세요. 설명 불필요.
"""

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "정산 문서 분류 전문가입니다."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,
                max_tokens=30
            )

            result = response.choices[0].message.content.strip()

            valid_types = {
                '이체확인증', '전자세금계산서', '원천징수영수증', '매출전표',
                '사업자등록증', '계약서', '견적서', '신분증', '통장사본',
                '이력서', '비용지급확인서', '자문보고서', '강의자료',
                '회의록', '방명록', '영수증', '기타'
            }

            return result if result in valid_types else '기타'

        except Exception as e:
            print(f"❌ GPT 파일 분류 오류: {e}")
            return '기타'

    def extract_folder_info(self, folder_name: str) -> dict:
        """폴더명에서 번호, 이름, 비용유형 추출 (GPT)"""
        self.call_count += 1

        prompt = f"""다음 폴더명에서 정보를 추출하세요.

폴더명: {folder_name}

추출할 정보:
1. 번호 (예: ["149"] 또는 ["149", "152"])
2. 업체/개인 이름 (예: "이정근", "문카데미주식회사", "(주)케이티앤지")
3. 비용 유형 (반드시 다음 중 하나: "일반수용비", "용역비", "행사비", "사업추진비", "국내여비", "기타")

JSON 형식으로만 답변:
{{
  "numbers": ["149"],
  "entity_name": "이정근",
  "expense_type": "일반수용비"
}}
"""

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "정산 폴더명 분석 전문가입니다."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,
                max_tokens=150
            )

            result = response.choices[0].message.content.strip()

            # JSON 파싱
            if result.startswith("```json"):
                result = result[7:]
            if result.startswith("```"):
                result = result[3:]
            if result.endswith("```"):
                result = result[:-3]

            data = json.loads(result.strip())

            valid_expense_types = {
                '일반수용비', '용역비', '행사비', '사업추진비', '국내여비', '기타'
            }

            if data.get('expense_type') not in valid_expense_types:
                data['expense_type'] = '기타'

            return data

        except Exception as e:
            print(f"❌ GPT 폴더 분석 오류: {e}")
            return {
                "numbers": None,
                "entity_name": None,
                "expense_type": "기타"
            }

# GPT Extractor 초기화
if Config.USE_GPT_CLASSIFICATION:
    gpt_extractor = GPTDocumentExtractor(Config.OPENAI_API_KEY)
    print("✅ GPT DocumentExtractor 초기화 완료 (파일 + 폴더 분류)")
    print(f"   API 키: {Config.OPENAI_API_KEY[:20]}...")
else:
    print("⚠️ 정규식 모드 (GPT 미사용)")


In [26]:
class FolderScanner:
    """폴더 스캔 및 문서 분류 (GPT 완전 통합)"""

    def __init__(self, base_path: str):
        self.base_path = Path(base_path)
        self.converter = FileConverter()

    def scan_all(self) -> List[Dict]:
        """모든 폴더 스캔"""
        results = []
        for folder in self.base_path.iterdir():
            if not folder.is_dir() or folder.name in ['주디네', '.claude']:
                continue
            folder_info = self.scan_folder(folder)
            if folder_info:
                results.append(folder_info)
        return results

    def scan_folder(self, folder_path: Path) -> Optional[Dict]:
        """폴더 스캔 및 파일 수집 (GPT로 전부 처리)"""
        folder_name = folder_path.name

        # PDF 파일 수집 (합본 제외)
        pdf_files = [p for p in folder_path.glob('*.pdf') if '합본' not in p.name]

        # 이미지 파일 수집
        image_files = []
        for ext in FileConverter.SUPPORTED_IMAGES:
            image_files.extend(folder_path.glob(f'*{ext}'))

        if not pdf_files and not image_files:
            return None

        # 임시 PDF 디렉토리 생성
        temp_pdf_dir = None
        if image_files:
            temp_pdf_dir = folder_path / '.temp_pdfs'
            temp_pdf_dir.mkdir(exist_ok=True)

        # 문서 분류
        documents = {}
        all_files = []
        conversion_log = []

        # 1. PDF 파일 처리 (GPT로 분류)
        for pdf in pdf_files:
            if Config.USE_GPT_CLASSIFICATION:
                doc_type = gpt_extractor.classify_file(pdf.name)
            else:
                # 정규식 폴백 (사용 안 함)
                doc_type = '기타'

            if doc_type not in documents:
                documents[doc_type] = []
            documents[doc_type].append({
                'filename': pdf.name,
                'path': str(pdf),
                'original_path': str(pdf),
                'is_converted': False
            })
            all_files.append(str(pdf))

        # 2. 이미지 파일 처리 (GPT로 분류)
        for img in image_files:
            temp_pdf_name = f"{img.stem}_converted.pdf"
            temp_pdf_path = temp_pdf_dir / temp_pdf_name

            success = self.converter.convert_image_to_pdf(img, temp_pdf_path)

            if success:
                if Config.USE_GPT_CLASSIFICATION:
                    doc_type = gpt_extractor.classify_file(img.name)
                else:
                    doc_type = '기타'

                if doc_type not in documents:
                    documents[doc_type] = []
                documents[doc_type].append({
                    'filename': img.name,
                    'path': str(temp_pdf_path),
                    'original_path': str(img),
                    'is_converted': True
                })
                all_files.append(str(temp_pdf_path))
                conversion_log.append({'file': img.name, 'status': 'success', 'type': doc_type})
            else:
                conversion_log.append({'file': img.name, 'status': 'failed', 'type': 'unknown'})

        # 폴더 정보 추출 (GPT)
        if Config.USE_GPT_CLASSIFICATION:
            folder_info = gpt_extractor.extract_folder_info(folder_name)
            numbers = folder_info.get('numbers')
            entity_name = folder_info.get('entity_name')
            expense_type = folder_info.get('expense_type')
        else:
            numbers = None
            entity_name = None
            expense_type = '기타'

        return {
            'folder_name': folder_name,
            'folder_path': str(folder_path),
            'numbers': numbers,
            'entity_name': entity_name,
            'expense_type': expense_type,
            'documents': documents,
            'total_files': len(pdf_files) + len(image_files),
            'pdf_files': all_files,
            'conversion_log': conversion_log,
            'temp_pdf_dir': str(temp_pdf_dir) if temp_pdf_dir else None
        }

    def get_summary(self, folder_info: Dict) -> str:
        """문서 요약 정보 생성"""
        parts = [f"{t}({len(fs)})" for t, fs in sorted(folder_info['documents'].items())]
        return ", ".join(parts)

    def cleanup_temp_files(self, folder_info: Dict):
        """임시 변환 파일 정리"""
        if folder_info.get('temp_pdf_dir'):
            temp_dir = Path(folder_info['temp_pdf_dir'])
            if temp_dir.exists():
                for f in temp_dir.glob('*_converted.pdf'):
                    try:
                        f.unlink()
                    except:
                        pass
                try:
                    temp_dir.rmdir()
                except:
                    pass

print("✅ FolderScanner (GPT 완전 통합) 정의 완료")


✅ FolderScanner (GPT 완전 통합) 정의 완료


In [27]:
class PDFMerger:
    """PDF 합본 생성"""

    def sort_by_order(self, folder_info: dict, doc_order: dict) -> List[str]:
        expense_type = folder_info['expense_type']
        documents = folder_info['documents']
        order = doc_order.get(expense_type, [])

        if not order:
            return folder_info['pdf_files']

        sorted_files = []
        for doc_type in order:
            if doc_type in documents:
                sorted_files.extend([f['path'] for f in documents[doc_type]])
        return sorted_files

    def merge(self, pdf_files: List[str], output_path: str) -> tuple:
        """PDF 병합 - 손상된 파일은 건너뜀"""
        try:
            merger = PdfMerger()
            success = []
            failed = []

            for pdf_file in pdf_files:
                try:
                    reader = PdfReader(pdf_file)
                    if len(reader.pages) > 0:
                        merger.append(pdf_file)
                        success.append(Path(pdf_file).name)
                except Exception as e:
                    failed.append(Path(pdf_file).name)

            if len(merger.pages) > 0:
                merger.write(output_path)
                merger.close()
                return (True, len(success), len(failed), len(merger.pages))
            return (False, 0, len(failed), 0)
        except Exception as e:
            print(f"❌ 오류: {e}")
            return (False, 0, len(pdf_files), 0)

    def merge_folder(self, folder_info: dict, doc_order: dict = None) -> Optional[str]:
        folder_path = Path(folder_info['folder_path'])

        # 순서대로 정렬
        pdf_files = self.sort_by_order(folder_info, doc_order) if doc_order else folder_info['pdf_files']

        if not pdf_files:
            return None

        # 파일명 생성
        numbers = folder_info.get('numbers', [])
        entity = folder_info.get('entity_name', '이름없음')
        expense = folder_info.get('expense_type', '기타')
        num_part = numbers[0] if numbers and len(numbers) == 1 else f"{numbers[0]}-{numbers[-1]}" if numbers else "000"

        filename = f"{num_part}_{entity}_{expense}_합본.pdf"
        for char in '<>:"|?*':
            filename = filename.replace(char, '_')

        output_path = folder_path / filename

        if output_path.exists():
            print(f"ℹ️  이미 존재: {filename}")
            return str(output_path)

        print(f"\n📁 {folder_info['folder_name']}")
        success, success_cnt, failed_cnt, total_pages = self.merge(pdf_files, str(output_path))

        if success:
            print(f"   ✅ {filename}")
            print(f"   성공: {success_cnt}/{len(pdf_files)}개 파일, {total_pages} 페이지")
            if failed_cnt > 0:
                print(f"   ⚠️  손상된 파일: {failed_cnt}개")
            return str(output_path)
        else:
            print(f"   ❌ 실패")
            return None

print("✅ PDFMerger 정의 완료")

✅ PDFMerger 정의 완료


## 4단계: 메인 실행 ▶️

**아래 셀을 실행하면 자동으로 모든 처리가 진행됩니다!**

In [28]:
# 메인 실행 - 실시간 스트리밍 로그
print("=" * 80)
print("정산 문서 자동 처리 시스템")
print("=" * 80)
print()

# 1. 폴더 스캔
print("[1/3] 폴더 스캔 중...")
scanner = FolderScanner(Config.BASE_PATH)
all_folders = scanner.scan_all()
print(f"      -> {len(all_folders)}개 폴더 발견")
print()

# 스캔 결과 미리보기 (처음 5개)
print("스캔 결과 미리보기:")
for i, folder_info in enumerate(all_folders[:5], 1):
    numbers = folder_info.get('numbers', [])
    num_str = numbers[0] if numbers and len(numbers) == 1 else f"{numbers[0]}-{numbers[-1]}" if numbers else "?"
    print(f"  [{num_str}] {folder_info.get('entity_name', '?')} ({folder_info['expense_type']})")
if len(all_folders) > 5:
    print(f"  ... 외 {len(all_folders)-5}개")
print()

# 2. 합본 생성 (실시간 스트리밍)
print(f"[2/3] PDF 합본 생성 중... (총 {len(all_folders)}개)")
print("-" * 80)
merger = PDFMerger()
merged_results = []

for i, folder_info in enumerate(all_folders, 1):
    numbers = folder_info.get('numbers', [])
    num_str = numbers[0] if numbers and len(numbers) == 1 else f"{numbers[0]}-{numbers[-1]}" if numbers else "000"
    entity = folder_info.get('entity_name', '?')
    expense = folder_info['expense_type']

    print(f"[{i}/{len(all_folders)}] {num_str}. {entity} ({expense})", end=" ")

    result = merger.merge_folder(folder_info, Config.DOCUMENT_ORDER)
    if result:
        merged_results.append({
            'folder': folder_info['folder_name'],
            'merged_file': result
        })
        print(f"✅ {Path(result).name}")
    else:
        print("❌ 실패")

    # 임시 파일 정리
    if Config.CLEANUP_TEMP_FILES:
        scanner.cleanup_temp_files(folder_info)

print()
print(f"합본 생성 완료: {len(merged_results)}/{len(all_folders)}개 성공")
print()

# 3. 결과 데이터 생성
print("[3/3] 결과 데이터 생성 중...")
data = [["폴더번호", "이름", "비용유형", "업로드파일", "필수문서체크", "누락문서", "합본파일명"]]

for folder_info in all_folders:
    numbers = folder_info.get('numbers', [])
    num_str = numbers[0] if numbers and len(numbers) == 1 else f"{numbers[0]}-{numbers[-1]}" if numbers else "없음"
    entity = folder_info.get('entity_name', '이름없음')
    expense_type = folder_info['expense_type']

    uploaded = scanner.get_summary(folder_info)

    required = Config.REQUIRED_DOCS.get(expense_type, [])
    existing = list(folder_info['documents'].keys())
    missing = [doc for doc in required if doc not in existing]

    check = "✅" if not missing else "❌"
    missing_str = ", ".join(missing) if missing else "없음"

    merged_file = next((m['merged_file'] for m in merged_results if m['folder'] == folder_info['folder_name']), "미생성")
    merged_filename = Path(merged_file).name if merged_file != "미생성" else "미생성"

    data.append([num_str, entity, expense_type, uploaded, check, missing_str, merged_filename])

print(f"      -> {len(data)-1}개 행 생성 완료")
print()

# 결과 요약
print("=" * 80)
print("처리 결과 요약 (처음 20개)")
print("=" * 80)
df = pd.DataFrame(data[1:], columns=data[0])
print(df.head(20).to_string(index=False))
print()
print("=" * 80)
print(f"✅ 완료! 총 {len(all_folders)}개 폴더 처리, {len(merged_results)}개 합본 생성")
print("=" * 80)

# API 사용량 표시
if Config.USE_GPT_CLASSIFICATION:
    print(f"\nGPT API 호출 횟수: {gpt_extractor.call_count}회")
    print(f"예상 비용: ${gpt_extractor.call_count * 0.00005:.4f} (GPT-4o-mini)")

In [30]:
# 구글 시트 업로드 (누적 업데이트 방식)
print("╔══════════════════════════════════════════════════════════════╗")
print("║              📤 구글 시트 업로드                              ║")
print("╚══════════════════════════════════════════════════════════════╝\n")

import gspread
from google.colab import auth
from google.auth import default

# Google 인증
print("🔐 Google 인증 중...")
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("  ✅ 인증 완료\n")

# 구글 시트 열기
print("📋 시트 열기 중...")
print(f"  URL: {Config.SHEET_URL}")
spreadsheet = gc.open_by_url(Config.SHEET_URL)
worksheet = spreadsheet.get_worksheet(0)
print(f"  ✅ 시트 열기 완료: {worksheet.title}\n")

# 기존 데이터 확인 (삭제 안 함)
print("📊 기존 데이터 확인 중...")
existing_data = worksheet.get_all_values()
if len(existing_data) > 1:
    print(f"  ℹ️  기존 데이터: {len(existing_data)-1}행 (유지됨)")
else:
    print(f"  ℹ️  기존 데이터: 없음")

# 새 데이터 추가 (헤더 + 데이터)
print(f"\n📝 새 데이터 업로드 중... ({len(data)}행)")

# 헤더가 없으면 추가
if len(existing_data) == 0:
    worksheet.update('A1', [data[0]], value_input_option='USER_ENTERED')
    start_row = 2
else:
    start_row = len(existing_data) + 1

# 데이터 추가 (헤더 제외)
if len(data) > 1:
    worksheet.update(f'A{start_row}', data[1:], value_input_option='USER_ENTERED')
    print(f"  ✅ 업로드 완료: {start_row}행부터 {len(data)-1}개 행 추가\n")
else:
    print(f"  ⚠️  추가할 데이터 없음\n")

# CSV 백업 저장
print("💾 CSV 백업 저장 중...")
df = pd.DataFrame(data[1:], columns=data[0])
output_csv = Path(Config.BASE_PATH) / "주디네" / f"처리결과_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"  ✅ 백업 완료: {output_csv.name}\n")

print("╔══════════════════════════════════════════════════════════════╗")
print("║              ✅ 구글 시트 업로드 완료!                        ║")
print("╚══════════════════════════════════════════════════════════════╝\n")
print(f"🔗 결과 확인: {Config.SHEET_URL}")
print(f"📊 이번 세션: {len(data)-1}개 행 추가")
print(f"📊 총 누적: {start_row + len(data) - 2}개 행")
print(f"💾 백업 파일: {output_csv.name}")